In [1]:
%pip install -qU langchain langchain_openai langchain-core langgraph langchain-community psycopg[binary,pool]==3.2.6 langgraph-checkpoint-postgres langchain-elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.3/906.3 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/4

In [2]:
import os
with open("/content/API_KEY_Openai.txt") as archivo:
  apikey = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey

with open("/content/postgres_txt.txt") as archivo:
  uribd = archivo.read()

In [3]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore

db_query= ElasticsearchStore(
    es_url="http://35.xx.61.2xxx:9200",
    es_user="elastic",
    es_password="claveelastisearch",
    index_name="elasticsportj-data",
    embedding=OpenAIEmbeddings())

retriever = db_query.as_retriever()

tool_rag =retriever.as_tool(
        name="busqueda_productos",
        description="Consulta en la informacion de implementos de artes marciales mixtas, running, levantamiento de pesas, suplementos deportivos, y medicina deportiva",
    )

<ipython-input-4-3939702670>:13: LangChainBetaWarning: This API is in beta and may change in the future.
  tool_rag =retriever.as_tool(


In [5]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.prebuilt import create_react_agent

In [6]:
## Ejecutar solo la primera vez para crear automaticamente las tablas
## [checkpoint_blobs, checkpoint_migrations, checkpoint_writes, checkpoints] en el esquema public
#Variables de memoria
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
# Inicializamos la memoria
with ConnectionPool(
    # Example configuration
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)
    checkpointer.setup()

In [7]:
#Variables de memoria
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}
# Inicializamos la memoria
with ConnectionPool(
    # Example configuration
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    # Inicializamos el modelo
    model = ChatOpenAI(model="gpt-4.1-2025-04-14")

    # Agrupamos las herramientas
    tolkit = [tool_rag]

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",
        """
        Eres un asesor experto en ventas deportivas, amable y entusiasta. Tu objetivo es guiar a los clientes en la compra de implementos para artes marciales mixtas, running, pesas, suplementos y medicina deportiva.

        Siempre utiliza las herramientas disponibles para obtener la información de productos (nombre, descripción, precio, stock). No inventes si no tienes información; en su lugar, guía con recomendaciones generales.

        Tu estilo debe ser conversacional, breve, directo y con un tono cálido, como un amigo que sabe mucho de deportes. Guía al cliente paso a paso:

        1. Saludo inicial:
           - Da una bienvenida cercana (ej. “¡Hola campeón! ¿Qué estás buscando hoy para tu rutina?”).
           - Pregunta si el cliente ya tiene en mente lo que necesita o si desea sugerencias.
           - Si no sabe, ofrece 2-3 opciones populares (por categoría), priorizando productos con más stock.

        2. Consulta de productos:
           - Filtra productos según lo que necesita el cliente.
           - Muestra nombre, breve descripción, precio y disponibilidad.
           - Si hay stock limitado, avisa. Si hay buen stock, anímalo con frases como “¡Aprovecha que está disponible!”.

        3. Envío o tienda:
           - Pregunta si quiere recoger en tienda o envío a domicilio.
           - Informa: Envío cuesta S/20 si la compra es menor a S/500. Si supera S/500, el envío es gratis.
           - Si está cerca del límite, sugiere productos complementarios útiles para llegar al envío gratis.

        4. Confirmación:
           - Resume el pedido actual y pregunta si quiere añadir algo más.

        5. Método de pago:
           - Si elige tienda, pregunta si pagará en efectivo o por transferencia.
           - Pide su nombre y apellido para generar un código de pedido (formato: AAAAMMDD_HHMMSS_NombreApellido).
           - Si elige domicilio, pide dirección completa y confirma pago por transferencia.

        6. Cierre:
           - Si es transferencia, da el número de cuenta: 12730317292820 en BankaNet y solicita confirmación.
           - Si paga en tienda, entrega el código de pedido y agradece su compra.

        7. Estilo:
           - Sé claro, breve y útil.
           - Usa expresiones como “¡Vamos con todo!”, “¡Listo campeón!”, “Perfecto, crack”.

        IMPORTANTE:
        - Siempre consulta a Elasticsearch antes de dar detalles de productos.
        - No hables como robot. Sé fresco, humano, motivador.
        """
        ),
        ("human", "{messages}"),
    ]
    )
    #inicializamos el agente
    agent_executor = create_react_agent(model, tolkit, checkpointer=checkpointer, prompt=prompt)


    config = {"configurable": {"thread_id": "abc201"}}
    for step in agent_executor.stream(
        {"messages": [HumanMessage(content="Estoy buscando proteina")]},
        config,
        stream_mode="values",
    ):
      step["messages"][-1].pretty_print()

================================ Human Message =================================

Estoy buscando proteina
================================== Ai Message ==================================
Tool Calls:
  busqueda_productos (call_FUDpBuMwZDsjPgultkpTTfwp)
 Call ID: call_FUDpBuMwZDsjPgultkpTTfwp
  Args:
    __arg1: proteína
================================= Tool Message =================================
Name: busqueda_productos

[Document(metadata={'source': '/content/Base_Tienda_Deportiva_AI.csv', 'row': 1271}, page_content='ID: 1272\nNombre: Proteína en polvo Pro 461\nCategoría: Suplementos deportivos\nSubcategoría: Proteína en polvo\nMarca: ProFit\nPrecio (USD): 13.01\nStock: 69\nPaís: España\nIdioma: español\nDescripción: Artículo de Proteína en polvo ideal para deportistas de nivel intermedio y avanzado.\nValoración: 3.3\nOpiniones: 483'), Document(metadata={'source': '/content/Base_Tienda_Deportiva_AI.csv', 'row': 1232}, page_content='ID: 1233\nNombre: Proteína en polvo Pro 199\nCateg